In [ ]:
import pandas as pd
import pandas as pd
from pathlib import Path

# File Paths
YEARLY_METRICS_FILE_PATH = "../data/preprocessed/ch2018/CH2018_yearly_metrics.csv"

ENSEMBLE_MEAN_OUTPUT_DIR = Path("../data/preprocessed/ch2018")

NFI_DATA_FILE_PATH = "../data/preprocessed/nfi/nfi_processed_data.csv"
CLOSEST_COORDS_FILE_PATH = "../data/preprocessed/NFI_CH2018_closest_coords.csv"
CH2018_PREPROCESSED_DIR = Path("../data/preprocessed/ch2018")
FINAL_NFI_CH2018_MERGED_DIR = Path("../data/preprocessed/final_nfi_ch2018_merged")



In [ ]:
# Load and inspect the yearly climate metrics data

try:
    # Load the CSV file into a DataFrame
    df_climate_metrics = pd.read_csv(YEARLY_METRICS_FILE_PATH)

    # Display Descriptives
    print(f"Successfully loaded data from: {YEARLY_METRICS_FILE_PATH}")
    print(f"Shape of the DataFrame: {df_climate_metrics.shape}\n")

    print("--- Head of the DataFrame ---")
    print(df_climate_metrics.head(100))

    print("\n--- DataFrame Info (Data Types and Non-Null Counts) ---")
    df_climate_metrics.info()

    
    print("\n--- Check for NaN values ---")
    print(df_climate_metrics.isnull().sum())


except FileNotFoundError:
    print(f"Error: File not found at the specified path: {YEARLY_METRICS_FILE_PATH}")
except Exception as e:
    print(f"An error occurred while loading or processing the file: {e}")

Successfully loaded data from: ../data/preprocessed/ch2018/CH2018_yearly_metrics.csv
Shape of the DataFrame: (33638444, 16)

--- Head of the DataFrame ---
    CLNR                        simulation  year  closest_CH2018_lon_4326  \
0      6  CLMCOM-CCLM4_ECEARTH_EUR11_RCP45  1981                   8.5833   
1      6  CLMCOM-CCLM4_ECEARTH_EUR11_RCP45  1982                   8.5833   
2      6  CLMCOM-CCLM4_ECEARTH_EUR11_RCP45  1983                   8.5833   
3      6  CLMCOM-CCLM4_ECEARTH_EUR11_RCP45  1984                   8.5833   
4      6  CLMCOM-CCLM4_ECEARTH_EUR11_RCP45  1985                   8.5833   
..   ...                               ...   ...                      ...   
95     6  CLMCOM-CCLM4_ECEARTH_EUR11_RCP45  2076                   8.5833   
96     6  CLMCOM-CCLM4_ECEARTH_EUR11_RCP45  2077                   8.5833   
97     6  CLMCOM-CCLM4_ECEARTH_EUR11_RCP45  2078                   8.5833   
98     6  CLMCOM-CCLM4_ECEARTH_EUR11_RCP45  2079                   8.5833  

In [3]:
# Calculate ensemble means for each RCP scenario and save them as separate files

print("--- Calculating Ensemble Means for each RCP Scenario (Separately) ---")

# --- Extract the RCP scenario from the 'simulation' column ---
df_climate_metrics['rcp'] = df_climate_metrics['simulation'].str.extract(r'(RCP\d{2})')

# Get a list of unique RCP scenarios found in the data
unique_rcps = df_climate_metrics['rcp'].unique()
print(f"Found the following scenarios to process: {unique_rcps}")
print (df_climate_metrics['CLNR'].nunique(), "unique CLNRs found in the data.")



# --- Define the columns to average ---
identifier_cols = ['CLNR', 'simulation', 'year', 'closest_CH2018_lon_4326', 'closest_CH2018_lat_4326', 'rcp']
metric_cols = [col for col in df_climate_metrics.columns if col not in identifier_cols]


# --- Loop through each RCP, create a separate DataFrame, and save it ---
# We will store the resulting dataframes in a dictionary
ensemble_dfs = {}

for rcp_scenario in unique_rcps:
    print(f"\n--- Processing {rcp_scenario} ---")
    
    # Filter the main DataFrame for the current scenario
    df_scenario = df_climate_metrics[df_climate_metrics['rcp'] == rcp_scenario].copy()
    print(f"  - Found {df_scenario.shape[0]} rows for {rcp_scenario}.")
    
    # Group by CLNR and year to get the ensemble mean for this scenario
    # Define aggregations
    aggregations = {col: 'mean' for col in metric_cols}
    aggregations['closest_CH2018_lon_4326'] = 'first'
    aggregations['closest_CH2018_lat_4326'] = 'first'

    df_ensemble_mean_scenario = df_scenario.groupby(['CLNR', 'year']).agg(aggregations).reset_index()
    print(f"  - Calculated ensemble mean. Resulting shape: {df_ensemble_mean_scenario.shape}")
    
    # Store the resulting dataframe in our dictionary
    ensemble_dfs[rcp_scenario] = df_ensemble_mean_scenario
    
    # Define the output filename and save the DataFrame to a CSV
    ENSMEMBLE_MEAN_OUTPUT_FILENAME = f"CH2018_ensemble_mean_multiple_vars_{rcp_scenario}.csv"
    output_path = ENSEMBLE_MEAN_OUTPUT_DIR / ENSMEMBLE_MEAN_OUTPUT_FILENAME
    
    print(f"  - Saving data to: {output_path}")
    df_ensemble_mean_scenario.to_csv(output_path, index=False, float_format='%.4f')

print("\n--- All scenarios processed and saved successfully! ---")

--- Calculating Ensemble Means for each RCP Scenario (Separately) ---
Found the following scenarios to process: ['RCP45' 'RCP85' 'RCP26']
4157 unique CLNRs found in the data.

--- Processing RCP45 ---
  - Found 12367075 rows for RCP45.
  - Calculated ensemble mean. Resulting shape: (494683, 15)
  - Saving data to: ../data/preprocessed/ch2018/CH2018_ensemble_mean_multiple_vars_RCP45.csv

--- Processing RCP85 ---
  - Found 15335173 rows for RCP85.
  - Calculated ensemble mean. Resulting shape: (494683, 15)
  - Saving data to: ../data/preprocessed/ch2018/CH2018_ensemble_mean_multiple_vars_RCP85.csv

--- Processing RCP26 ---
  - Found 5936196 rows for RCP26.
  - Calculated ensemble mean. Resulting shape: (494683, 15)
  - Saving data to: ../data/preprocessed/ch2018/CH2018_ensemble_mean_multiple_vars_RCP26.csv

--- All scenarios processed and saved successfully! ---


In [4]:
# Load and inspect the processed NFI data.

df_nfi = pd.read_csv(NFI_DATA_FILE_PATH)
print(f"Successfully loaded NFI data from: {NFI_DATA_FILE_PATH}")
print(f"Shape of the NFI DataFrame: {df_nfi.shape}\n")

print("--- Head of the NFI DataFrame ---")
print(df_nfi.head(15))

print("\n--- NFI DataFrame Info (Data Types and Non-Null Counts) ---")
df_nfi.info()

print(df_nfi['CLNR'].nunique(), "unique CLNRs found in the NFI data.")
clnrs_nfi = df_nfi['CLNR'].unique()
clnrs = df_climate_metrics['CLNR'].unique()
# Check if the CLNRs in the NFI data match those in the climate metrics data
if set(clnrs) == set(clnrs_nfi):
    print("All CLNRs match between the NFI data and the climate metrics data.")
else:
    print("There are mismatches in CLNRs between the NFI data and the climate metrics data.")
    missing_in_nfi = set(clnrs) - set(clnrs_nfi)
    missing_in_climate = set(clnrs_nfi) - set(clnrs)
    if missing_in_nfi:
        print(f"#CLNRs in climate metrics but not in NFI: {len(missing_in_nfi)}")
    if missing_in_climate:
        print(f"#CLNRs in NFI but not in climate metrics: {len(missing_in_climate)}")
# 77 are in the climate metrics but not in the NFI data, because i changed the /data/preprocessed/NFI_CH2018_closest_coords.csv file but i didnt rerun the yearly climate data calculations on euler with the updated file.
# But it does not matter, because some CLNRs are in climate data but not in NFI and not the other way around. 
# (In the final model i even used less CLNRs so i could have furhter updated the /data/preprocessed/NFI_CH2018_closest_coords.csv file and ran the scripts/prediction_until_2099.py on even less coordinates.)

Successfully loaded NFI data from: ../data/preprocessed/nfi/nfi_processed_data.csv
Shape of the NFI DataFrame: (51872, 21)

--- Head of the NFI DataFrame ---
    CLNR      DATUMF  INVNR  BASFPH  BEWIRTINT1    ASPECT25    SLOPE25    PH  \
0      6  16.04.1984    150   23.85         1.0  104.901916  31.302792  7.21   
1      6  19.10.1994    250   48.99         2.0  104.901916  31.302792  7.21   
2      6  01.11.2005    350   37.91         1.0  104.901916  31.302792  7.21   
3      6  24.04.2015    450   15.73         1.0  104.901916  31.302792  7.21   
4      6  23.10.2024    550   12.09         1.0  104.901916  31.302792  7.21   
5      6         NaN    650     NaN         1.0  104.901916  31.302792  7.21   
6      6         NaN    750     NaN         1.0  104.901916  31.302792  7.21   
7      6         NaN    850     NaN         1.0  104.901916  31.302792  7.21   
8      6         NaN    950     NaN         1.0  104.901916  31.302792  7.21   
9      6         NaN   1050     NaN       

In [5]:
# Merge NFI data with the averaged climate data for each inventory interval


# --- Load and Prepare NFI Data ---
print("--- Loading and preparing NFI data ---")
df_nfi = pd.read_csv(NFI_DATA_FILE_PATH)
df_coords = pd.read_csv(CLOSEST_COORDS_FILE_PATH)

# Add the CH2018 coordinates to the NFI dataframe
coords_to_add = df_coords[['CLNR', 'closest_CH2018_lon_4326', 'closest_CH2018_lat_4326']].drop_duplicates(subset=['CLNR'])
df_nfi_with_coords = pd.merge(df_nfi, coords_to_add, on='CLNR', how='left')
print(f"NFI data prepared. Shape: {df_nfi_with_coords.shape}")


# --- Load Climate Data ---
print("\n--- Loading all ensemble-mean climate data ---")
climate_dfs = {}
rcp_scenarios = ["RCP26", "RCP45", "RCP85"]
for rcp in rcp_scenarios:
    file = CH2018_PREPROCESSED_DIR / f"CH2018_ensemble_mean_multiple_vars_{rcp}.csv"
    climate_dfs[rcp] = pd.read_csv(file)
    print(f"Loaded {rcp} data with shape: {climate_dfs[rcp].shape}")

# Identify the columns we need to average
climate_metric_cols = [
    'dry_days_count', 'frost_days_count', 'gdd_sum', 'pr_sum', 'pr_variance',
    'tas_mean', 'tas_variance', 'tasmax_mean', 'tasmax_variance',
    'tasmin_mean', 'tasmin_variance'
]

# --- Process Each RCP Scenario ---
for rcp in rcp_scenarios:
    print(f"\n--- Processing Scenario: {rcp} ---")
    
    # Get the correct climate dataframe for this scenario
    df_climate = climate_dfs[rcp]

    # - Create the expanded lookup table from NFI data -
    print("  - Expanding NFI data to create a yearly lookup table...")
    
    # We only need intervals where we have a time difference
    nfi_intervals = df_nfi_with_coords.dropna(subset=['Time_Diff_years'])
    
    expanded_rows = []
    for _, row in nfi_intervals.iterrows():
        # These columns identify the original NFI record
        clnr = row['CLNR']
        invyr = int(row['INVYR'])
        
        # These columns are for the join and interval calculation
        lon = row['closest_CH2018_lon_4326']
        lat = row['closest_CH2018_lat_4326']
        time_diff = int(row['Time_Diff_years'])
        
        # Create a row for each year in the interval
        for i in range(time_diff):
            year_to_match = invyr + i
            expanded_rows.append({
                'CLNR': clnr,
                'INVYR': invyr, # Keep original start year to group by later
                'closest_CH2018_lon_4326': lon,
                'closest_CH2018_lat_4326': lat,
                'year_to_match': year_to_match
            })

    df_expanded_lookup = pd.DataFrame(expanded_rows)

    # - Merge -
    print("  - Merging expanded NFI data with yearly climate data")

    # The climate data already has CLNR, which is the unique ID for a plot.
    # We use it for the merge.
    climate_join_cols = ['CLNR', 'year'] + climate_metric_cols 

    # In df_expanded_lookup, 'year_to_match' is the climate year.
    df_merged_yearly = pd.merge(
        df_expanded_lookup, # Contains CLNR, INVYR, year_to_match
        df_climate[climate_join_cols],
        left_on=['CLNR', 'year_to_match'],
        right_on=['CLNR', 'year'],
        how='left'
    )
    # We can drop the redundant 'year' column from the merge
    df_merged_yearly.drop(columns=['year'], inplace=True)

    # - Group by original interval and calculate the mean -
    print("  - Calculating the mean of climate metrics over each interval...")
    # Group by the original unique record identifier: (CLNR, INVYR)
    df_averaged_climate = df_merged_yearly.groupby(['CLNR', 'INVYR'])[climate_metric_cols].mean().reset_index()
    
    # Rename columns to indicate they are interval averages
    df_averaged_climate.rename(columns={col: f'mean_{col}' for col in climate_metric_cols}, inplace=True)
    
    # - Merge the averaged climate data back into the main NFI dataframe -
    print("  - Merging averaged climate data back into the main NFI dataframe...")
    df_final_nfi_climate = pd.merge(
        df_nfi_with_coords,
        df_averaged_climate,
        on=['CLNR', 'INVYR'],
        how='left'
    )
    
    # - Save the final result to a new CSV file -
    output_path = FINAL_NFI_CH2018_MERGED_DIR / f"NFI_with_Climate_Averages_{rcp}.csv"
    print(f"  - Saving final data to: {output_path}")
    df_final_nfi_climate.to_csv(output_path, index=False, float_format='%.4f')
    
    print(f"  - Finished processing {rcp}. Final shape: {df_final_nfi_climate.shape}")
    # Display head of the new columns for verification
    new_cols = ['CLNR', 'INVYR', 'Time_Diff_years'] + list(df_averaged_climate.columns)[2:]
    print("    Verification of new columns:")
    print(df_final_nfi_climate[df_final_nfi_climate['mean_gdd_sum'].notna()][new_cols].head())

print("\n--- All scenarios have been processed successfully! ---")

--- Loading and preparing NFI data ---
NFI data prepared. Shape: (51872, 23)

--- Loading all ensemble-mean climate data ---
Loaded RCP26 data with shape: (494683, 15)
Loaded RCP45 data with shape: (494683, 15)
Loaded RCP85 data with shape: (494683, 15)

--- Processing Scenario: RCP26 ---
  - Expanding NFI data to create a yearly lookup table...
  - Merging expanded NFI data with yearly climate data
  - Calculating the mean of climate metrics over each interval...
  - Merging averaged climate data back into the main NFI dataframe...
  - Saving final data to: ../data/preprocessed/final_nfi_ch2018_merged/NFI_with_Climate_Averages_RCP26.csv
  - Finished processing RCP26. Final shape: (51872, 34)
    Verification of new columns:
   CLNR  INVYR  Time_Diff_years  mean_dry_days_count  mean_frost_days_count  \
0     6   1984             10.0           231.166660             112.683350   
1     6   1994             11.0           232.348491             105.848482   
2     6   2005             1

In [6]:
# Validate the final merged NFI-climate files to ensure data integrity

# --- Configuration ---

rcp_scenarios = ["RCP26", "RCP45", "RCP85"]

# The list of columns that should have been newly added and calculated
expected_new_climate_cols = [
    'mean_dry_days_count', 'mean_frost_days_count', 'mean_gdd_sum', 
    'mean_pr_sum', 'mean_pr_variance', 'mean_tas_mean', 'mean_tas_variance',
    'mean_tasmax_mean', 'mean_tasmax_variance', 'mean_tasmin_mean', 
    'mean_tasmin_variance'
]

print("--- Starting Validation of Final Merged NFI-Climate Files ---")

# --- Loop Through and Validate Each File ---
for rcp in rcp_scenarios:
    file_path = FINAL_NFI_CH2018_MERGED_DIR / f"NFI_with_Climate_Averages_{rcp}.csv"
    
    print(f"\n\n{'='*20} Validating: {file_path.name} {'='*20}")
    
    # --- Check 1: File Existence and Loading ---
    if not file_path.exists():
        print(f"ERROR: File not found! Please check the path and filename.")
        continue # Skip to the next file
        
    try:
        df = pd.read_csv(file_path)
        print(f"Successfully loaded file. Shape: {df.shape}")
    except Exception as e:
        print(f"ERROR: Could not read file. Error: {e}")
        continue

    # --- Check 2: Column Integrity ---
    missing_cols = [col for col in expected_new_climate_cols if col not in df.columns]
    if not missing_cols:
        print("SUCCESS: All expected new climate columns are present in the DataFrame.")
    else:
        print(f"ERROR: The following expected columns are MISSING: {missing_cols}")
        continue # Can't proceed with NaN check if columns are missing

    # --- Check 3: In-Depth NaN Analysis ---
    print("\n--- NaN Value Analysis ---")
    
    # General overview of NaNs in all columns
    nan_counts = df.isnull().sum()
    nan_counts = nan_counts[nan_counts > 0] # Filter to show only columns with NaNs
    if not nan_counts.empty:
        print("Overall NaN counts per column (only columns with NaNs are shown):")
        print(nan_counts.to_string())
    else:
        print("No NaNs found in any column.")

    # Focused check on the new climate columns
    nans_in_new_cols = df[expected_new_climate_cols].isnull().any(axis=1).sum()
    
    if nans_in_new_cols == 0:
        print("\nSUCCESS: Found 0 NaN values in the newly added climate columns. This is excellent.")
    else:
        print(f"\nINFO: Found {nans_in_new_cols} rows with NaN values in the new climate columns.")
        
        # Verify that these NaNs are expected
        # Get rows where the new climate data is NaN
        rows_with_nan_climate = df[df['mean_gdd_sum'].isnull()]
        
        # Check if the 'Time_Diff_years' column is also NaN in those same rows
        time_diff_nan_count = rows_with_nan_climate['Time_Diff_years'].isnull().sum()
        
        if time_diff_nan_count == nans_in_new_cols:
            print("SUCCESS: The NaNs in the new climate columns are all in rows where 'Time_Diff_years' was not defined.")
            print("         This is the correct and expected behavior.")
        else:
            print("ERROR: There is a mismatch. Some climate data is missing even for rows that had a valid time interval.")

    # --- Check 4: Data Plausibility ---
    print("\n--- Data Plausibility Check (Summary Statistics) ---")
    
    # Describe a few key new columns to spot obvious errors
    key_cols_to_describe = ['mean_gdd_sum', 'mean_pr_sum', 'mean_tas_mean']
    
    # Use a try-except block in case a column doesn't exist
    try:
        # We use .loc to select rows where the data is not null for a more accurate description
        description = df.loc[df['mean_gdd_sum'].notna(), key_cols_to_describe].describe()
        print("Summary for key new climate variables (for rows with data):")
        print(description)
    except KeyError:
        print("Could not generate summary statistics because one of the key columns was not found.")

print(f"\n\n{'='*20} Validation Complete {'='*20}")

--- Starting Validation of Final Merged NFI-Climate Files ---


==================== Validating: NFI_with_Climate_Averages_RCP26.csv ====================
Successfully loaded file. Shape: (51872, 34)
SUCCESS: All expected new climate columns are present in the DataFrame.

--- NaN Value Analysis ---
Overall NaN counts per column (only columns with NaNs are shown):
DATUMF                   32152
BASFPH                   32152
HW                       32152
SW                       32152
HWSW_prop                32152
BASFPH_squared           32152
Time_Diff_years           4080
BASFPH_next_INVNR        36232
HWSW_prop_next_INVNR     36232
BASFPH_difference        36232
mean_dry_days_count       4080
mean_frost_days_count     4080
mean_gdd_sum              4080
mean_pr_sum               4080
mean_pr_variance          4080
mean_tas_mean             4080
mean_tas_variance         4080
mean_tasmax_mean          4080
mean_tasmax_variance      4080
mean_tasmin_mean          4080
mean_tasmin_var